In [5]:
!pip install wikipedia-api

In [6]:
import wikipediaapi

wiki_wiki = wikipediaapi.Wikipedia('MyProjectName', 'en',
        extract_format=wikipediaapi.ExtractFormat.WIKI
)

p_wiki = wiki_wiki.page("Breakfast")
text = p_wiki.text
print('text: ', text)

text:  Breakfast is the first meal of the day usually eaten in the morning. The word in English refers to breaking the fasting period of the previous night. Various "typical" or "traditional" breakfast menus exist, with food choices varying by regions and traditions worldwide.

History
In Old English, a regular morning meal was called morgenmete, and the word dinner, which originated from Gallo-Romance desjunare ("to break one's fast"), referred to a meal after fasting. Around the mid-13th century, that meaning of dinner faded away, and around the 15th century "breakfast" came into use in written English to describe a morning meal.

Ancient breakfast
Ancient Egypt
In Ancient Egypt, peasants ate a daily meal, most likely in the morning, consisting of soup, beer, bread, and onions before they left for work in the fields or work commanded by the pharaohs.
The traditional breakfast believed to have been cooked in ancient Egypt was fūl (made from fava beans, possibly the ancestor of today's

In [7]:
import re

def text_2_sen(text):
    # A basic sentence tokenizer
    sentences = re.split(r'(?<=[.!?])\s+', text.strip())
    return sentences

def remove_text_from_start_end_marker(text, start_marker='(', end_marker=')'):
    # Remove parentheses and their content
    return re.sub(r'\{}.*?\{}'.format(re.escape(start_marker), re.escape(end_marker)), '', text).strip()

def convert_text2_sentences(text):

    sen = text_2_sen(text)
    # print('sen: ', sen)

    # --------------------------------
    # Add additional made-up sentences
    # --------------------------------
    sen.append('for breakfast it is typical to drink orange juice and eat eggs')
    sen.append('for breakfast the most common drink for breakfast is orange juice')
    sen.append('breakfast time is often from sunrise to a few hours before lunchtime')
    sen.append('for breakfast the most widely sold and bought drink is orange juice and milk')
    # print('sen: ', sen)

    # --------------------------------
    # Clean the sentences
    # --------------------------------
    # Make sentences lowercase
    sen1 = [i.lower() for i in sen]

    # Remove parentheses and text in between parentheses
    sen2 = [remove_text_from_start_end_marker(i) for i in sen1]
    # print('sen2: ', sen2)

    # Split the sentences on a comma to make shorter sentences
    sen3 = []
    for i in sen2:
        temp = i.split(',')
        for j in temp:
            sen3.append(j.strip())
    # print('sen3: ', sen3)

    # Remove undesirable characters
    to_replace = ["!", ";", '\n', '</p>', '<a', 'id=', "href=", 'title=', 'class=', '</a>', '(', ')', '}', '{',
                  '</sup>', '<p>', '</b>', '<sup', '>', '<', '\\', '-']
    replace_with = ''

    sen4 = []
    for i in sen3:
        word_array = i.split()
        word_array_new = []
        for word in word_array:
            for to_replace_val in to_replace:
                word = word.replace(to_replace_val, replace_with)
            word_array_new.append(word)
        cleaned_sentence = ' '.join(word_array_new).strip()
        # Remove extra whitespaces
        cleaned_sentence = re.sub(r'\s+', ' ', cleaned_sentence)
        sen4.append(cleaned_sentence)
    # print('sen4: ', sen4)

    # --------------------------------
    # Remove sentences with less than 10 words.
    # --------------------------------
    sen5 = [i for i in sen4 if len(i.split()) > 10]
    # print('sen5: ', sen5)

    return sen5

# Example usage:
sentences = convert_text2_sentences(text)

In [8]:
from collections import defaultdict, Counter
from transformers import AutoTokenizer

# Load Hugging Face tokenizer (you can change the model name!)
tokenizer = AutoTokenizer.from_pretrained("bert-base-uncased")

def compute_next_token_probabilities(sentences, given_token_text):
    # Tokenize all sentences into token ids
    tokens = []
    for sentence in sentences:
        token_ids = tokenizer.encode(sentence, add_special_tokens=False)
        tokens.extend(token_ids)

    # Convert the given token text to token id
    given_token_id = tokenizer.convert_tokens_to_ids(given_token_text)
    if given_token_id == tokenizer.unk_token_id:
        print(f"Warning: '{given_token_text}' is unknown token in tokenizer.")

    # Dictionary to store next-token counts
    next_token_counts = defaultdict(Counter)

    # Populate next-token counts
    for current_token, next_token in zip(tokens[:-1], tokens[1:]):
        next_token_counts[current_token][next_token] += 1

    # Calculate probabilities
    total_next = sum(next_token_counts[given_token_id].values())
    if total_next == 0:
        return {}

    probabilities = {
        tokenizer.convert_ids_to_tokens(token_id): count / total_next
        for token_id, count in next_token_counts[given_token_id].items()
    }

    return probabilities

# Example usage:
given_token_text = 'breakfast'  # Note: this must be the **token** as string, not id
probabilities = compute_next_token_probabilities(sentences, given_token_text)

# Output probabilities
for next_token, prob in sorted(probabilities.items(), key=lambda x: x[1], reverse=True):
    print(f"'{given_token_text}' → '{next_token}': {prob:.2f}")


/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


tokenizer_config.json:   0%|          | 0.00/48.0 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/570 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/466k [00:00<?, ?B/s]

'breakfast' → 'cereal': 0.13
'breakfast' → 'is': 0.11
'breakfast' → '.': 0.07
'breakfast' → '"': 0.04
'breakfast' → 'in': 0.04
'breakfast' → 'it': 0.04
'breakfast' → 'was': 0.04
'breakfast' → 'dish': 0.04
'breakfast' → ':': 0.04
'breakfast' → 'the': 0.04
'breakfast' → 'believed': 0.02
'breakfast' → 'as': 0.02
'breakfast' → 'when': 0.02
'breakfast' → 'cuisine': 0.02
'breakfast' → 'usually': 0.02
'breakfast' → 'drinks': 0.02
'breakfast' → 'drink': 0.02
'breakfast' → 'came': 0.02
'breakfast' → 'often': 0.02
'breakfast' → 'sandwich': 0.02
'breakfast' → 'beverage': 0.02
'breakfast' → 'high': 0.02
'breakfast' → 'q': 0.02
'breakfast' → 'consumption': 0.02
'breakfast' → 'eating': 0.02
'breakfast' → 'illustrations': 0.02
'breakfast' → 'time': 0.02


In [10]:
from collections import defaultdict, Counter
from transformers import AutoTokenizer

# Load tokenizer
tokenizer = AutoTokenizer.from_pretrained("bert-base-uncased")

def compute_next_token_counts(tokens):
    next_token_counts = defaultdict(Counter)
    for current_token, next_token in zip(tokens[:-1], tokens[1:]):
        next_token_counts[current_token][next_token] += 1
    return next_token_counts

def prepare_token_data(sentences):
    # Tokenize all sentences into token ids
    tokens = []
    for sentence in sentences:
        token_ids = tokenizer.encode(sentence, add_special_tokens=False)
        tokens.extend(token_ids)
    return tokens

def greedy_generate_sentence(sentences, start_token_text, max_length=20):
    tokens = prepare_token_data(sentences)
    next_token_counts = compute_next_token_counts(tokens)

    # Start token
    current_token_id = tokenizer.convert_tokens_to_ids(start_token_text)
    if current_token_id == tokenizer.unk_token_id:
        print(f"Warning: '{start_token_text}' is unknown in the tokenizer vocabulary.")

    generated_tokens = [current_token_id]

    for _ in range(max_length):
        # Get next token counts
        next_counts = next_token_counts.get(current_token_id, None)
        if not next_counts:
            break  # No next token found

        # Greedily pick the most probable next token
        next_token_id = next_counts.most_common(1)[0][0]
        generated_tokens.append(next_token_id)

        # Update current token
        current_token_id = next_token_id

        # Optional: break if punctuation token or special token is reached
        token_text = tokenizer.convert_ids_to_tokens(current_token_id)
        if token_text in ['.', '!', '?', tokenizer.sep_token, tokenizer.pad_token]:
            break

    # Convert token ids back to text
    generated_text = tokenizer.convert_tokens_to_string(tokenizer.convert_ids_to_tokens(generated_tokens))
    return generated_text

# Example usage:
start_token_text = 'breakfast'
generated_sentence = greedy_generate_sentence(sentences, start_token_text)
print("Generated sentence:")
print(generated_sentence)


Generated sentence:
breakfast cereals.


In [ ]:
import random
from collections import defaultdict, Counter
from transformers import AutoTokenizer

# Load tokenizer
tokenizer = AutoTokenizer.from_pretrained("bert-base-uncased")

def compute_next_token_counts(tokens):
    next_token_counts = defaultdict(Counter)
    for current_token, next_token in zip(tokens[:-1], tokens[1:]):
        next_token_counts[current_token][next_token] += 1
    return next_token_counts

def prepare_token_data(sentences):
    # Tokenize all sentences into token ids
    tokens = []
    for sentence in sentences:
        token_ids = tokenizer.encode(sentence, add_special_tokens=False)
        tokens.extend(token_ids)
    return tokens

def sample_next_token(next_counts):
    tokens, counts = zip(*next_counts.items())
    total = sum(counts)
    probabilities = [count / total for count in counts]
    return random.choices(tokens, weights=probabilities, k=1)[0]

def random_sample_generate_sentence(sentences, start_token_text, max_length=20):
    tokens = prepare_token_data(sentences)
    next_token_counts = compute_next_token_counts(tokens)

    # Start token
    current_token_id = tokenizer.convert_tokens_to_ids(start_token_text)
    if current_token_id == tokenizer.unk_token_id:
        print(f"Warning: '{start_token_text}' is unknown in the tokenizer vocabulary.")

    generated_tokens = [current_token_id]

    for _ in range(max_length):
        # Get next token counts
        next_counts = next_token_counts.get(current_token_id, None)
        if not next_counts:
            break  # No next token found

        # Sample next token from distribution
        next_token_id = sample_next_token(next_counts)
        generated_tokens.append(next_token_id)

        # Update current token
        current_token_id = next_token_id

        # Optional: stop if punctuation token or special token
        token_text = tokenizer.convert_ids_to_tokens(current_token_id)
        if token_text in ['.', '!', '?', tokenizer.sep_token, tokenizer.pad_token]:
            break

    # Convert token ids back to text
    generated_text = tokenizer.convert_tokens_to_string(tokenizer.convert_ids_to_tokens(generated_tokens))
    return generated_text

# Example usage:
start_token_text = 'breakfast'
generated_sentence = random_sample_generate_sentence(sentences, start_token_text)
print("Generated sentence (random sample):")
print(generated_sentence)


Generated sentence (random sample):
breakfast cereals were made from roasted spelt wheat or barley that was created by anita stewart that was un
